In [344]:
import pycaret `
import pandas as pd
import matplotlib.pyplot as plt

SyntaxError: invalid syntax (3412943160.py, line 1)

## 1 Data Engineering

In [345]:
# step 1
data = pd.read_csv(r"/Users/user/Downloads/10_RapidKL_Hackthon/data_model.csv")

In [346]:
print('Total size : ',data.shape)

Total size :  (14678, 50)


In [347]:
print('The columsn :',data.columns)

The columsn : Index(['ope_date', 'depot_id', 'line_id_x', 'bus_no', 'point_a', 'point_b',
       'ttl_td_id', 'ttl_captain_id', 'ttl_point_a', 'ttl_point_b',
       'max_diff_minutes_from_point_a', 'max_diff_minutes_idle_mode',
       'max_diff_minutes_to_point_a', 'min_diff_minutes_from_point_a',
       'min_diff_minutes_idle_mode', 'min_diff_minutes_to_point_a',
       'avg_diff_minutes_from_point_a', 'avg_diff_minutes_idle_mode',
       'avg_diff_minutes_to_point_a', 'med_diff_minutes_from_point_a',
       'med_diff_minutes_idle_mode', 'med_diff_minutes_to_point_a',
       'max_speed', 'min_speed', 'avg_speed', 'med_speed', 'ttl_td_completed',
       'line_id_y', 'br_id', 'br_repair_status', 'br_entry_status', 'br_date',
       'br_date_entry', 'br_target_completed', 'br_ttl_days', 'rtd_id',
       'rtd_date', 'route_id', 'bus_model_id', 'driver_id', 'rtd_cause',
       'cause_remark', 'fc_remark', 'date_issues', 'capt_id', 'bcc_acc_id',
       'acc_date', 'bus_damage', 'eng_find', 

In [ ]:
# #  description of the columns
# #
# # trip_departure.csv
# 'ope_date'
# 'depot_id'
# 'line_id_x'
# 'bus_no'
# 'point_a'
# 'point_b'

# # accident.csv 
# 'ttl_captain_id'
# 'ttl_point_a'
# 'ttl_point_b'

# # ??
# 'max_diff_minutes_from_point_a'
# 'max_diff_minutes_idle_mode'

# 'max_diff_minutes_to_point_a'
# 'min_diff_minutes_from_point_a'

# 'min_diff_minutes_idle_mode'
# 'min_diff_minutes_to_point_a'

# 'avg_diff_minutes_from_point_a'
# 'avg_diff_minutes_idle_mode'

# 'avg_diff_minutes_to_point_a'
# 'med_diff_minutes_from_point_a'

# 'med_diff_minutes_idle_mode'
# 'med_diff_minutes_to_point_a'

# # ??
# 'max_speed'
# 'min_speed'
# 'avg_speed'
# 'med_speed'
# 'ttl_td_completed'

# 'line_id_y'

# # eng_bus_repair.csv
# 'br_id'
# 'br_repair_status'
# 'br_entry_status'
# 'br_date'
# 'br_date_entry'
# 'br_target_completed'
# 'br_ttl_days'
# # return to depot
# # rtd.csv
# 'rtd_id'
# 'rtd_date'
# 'route_id'
# 'bus_model_id'
# 'driver_id'
# 'rtd_cause'
# 'cause_remark'
# 'fc_remark'
# 'date_issues'
# 'capt_id'
# 'bcc_acc_id'
# # accident.csv 
# 'acc_date'
# 'bus_damage'
# 'eng_find'
# 'label'

In [348]:
# step 2
# should drop this since we are not sure how to process this remarks 
data = data.drop(['cause_remark'], axis=1)
data = data.drop(['fc_remark'], axis=1)
data = data.drop(['bus_damage'], axis=1)
# should drop since only 1 or less value or banyak null value 
data = data.drop(['acc_date'], axis=1)
data = data.drop(['eng_find'], axis=1)
data = data.drop(['bcc_acc_id'], axis=1)

In [ ]:
# columnms di bawah hendak digunakan 
# sebab nya columns di bawah sudah dalam numerical 
#data = data.drop(['line_id_x'], axis=1)
#data = data.drop(['rtd_date'], axis=1)
#data = data.drop(['rtd_cause'], axis=1)
#data = data.drop(['br_id'], axis=1)
#data = data.drop(['br_repair_status'], axis=1)
#data = data.drop(['br_entry_status'], axis=1)
#data = data.drop(['br_ttl_days'], axis=1)
#data = data.drop(['rtd_id'], axis=1)
#data = data.drop(['route_id'], axis=1)
#data = data.drop(['bus_model_id'], axis=1)
#data = data.drop(['driver_id'], axis=1)
#data = data.drop(['capt_id'], axis=1)

In [349]:
# step 3 
# columns ini semua adalah dates
# kena make sense dulu 

# do conversion
#
#
#
data['ope_date'] = pd.to_datetime(data['ope_date'],format='%Y-%m-%d')
data['date_issues'] = pd.to_datetime(data['date_issues'],format='%Y-%m-%d')
data['br_date'] = pd.to_datetime(data['br_date'],format='%Y-%m-%d')
data['br_date_entry'] = pd.to_datetime(data['br_date_entry'],format='%Y-%m-%d')
data['br_target_completed'] = pd.to_datetime(data['br_target_completed'],format='%Y-%m-%d')

# berapa beza waktu masa antara ope_date(trip) dengan mula tanda ada problem(date_issues)
data['WaktuSebelumProblem'] = data['ope_date'].dt.day-data['date_issues'].dt.day
# berapa beza waktu masa antara bus repair(sepatutnya) dengan masa dilakukan inspection (br_date_entry)
data['WaktuSebelumInspection'] = data['br_date_entry'].dt.day-data['br_date'].dt.day
# berapa beza waktu masa digunakan untuk repair antara br_date_entry dengan br_target_completed
data['WaktuDiambikUtkRepair'] = data['br_target_completed'].dt.day-data['br_date_entry'].dt.day
# 

# once done, the tdrop
# 
data = data.drop(['ope_date'], axis=1)
data = data.drop(['date_issues'], axis=1)
data = data.drop(['br_date'], axis=1)
data = data.drop(['br_date_entry'], axis=1)
data = data.drop(['br_target_completed'], axis=1)

In [350]:
# Step 4 
# baru encoding yg belum numerical nie. 
# encode 1 
# bus label
data.cc = pd.Categorical(data.label)
data['labelencoded'] = data.cc.codes
data = data.drop('label',axis=1)
print(data.labelencoded.value_counts())
# encode 2 
# bus id 
data.cc = pd.Categorical(data.bus_no)
data['busencoded'] = data.cc.codes
data = data.drop('bus_no',axis=1)
print(data.busencoded.value_counts())
# encode 3
# point_a
data.cc = pd.Categorical(data.point_a)
data['point_aencoded'] = data.cc.codes
data = data.drop('point_a',axis=1)
print(data.point_aencoded.value_counts())
# encode 4
# point_b
data.cc = pd.Categorical(data.point_b)
data['point_bencoded'] = data.cc.codes
data = data.drop('point_b',axis=1)
print(data.point_bencoded.value_counts())
# encode 5
# line_id_y
data.cc = pd.Categorical(data.line_id_y)
data['line_id_yencoded'] = data.cc.codes
data = data.drop('line_id_y',axis=1)
print(data.line_id_yencoded.value_counts())
# encode 6
# br_id
data.cc = pd.Categorical(data.br_id)
data['br_idencoded'] = data.cc.codes
data = data.drop('br_id',axis=1)
print(data.br_idencoded.value_counts())
# encode 7
# br_id
data.cc = pd.Categorical(data.br_repair_status)
data['br_repair_statusencoded'] = data.cc.codes
data = data.drop('br_repair_status',axis=1)
print(data.br_repair_statusencoded.value_counts())
# encode 8
# br_id
data.cc = pd.Categorical(data.br_entry_status)
data['br_entry_statusencoded'] = data.cc.codes
data = data.drop('br_entry_status',axis=1)
print(data.br_entry_statusencoded.value_counts())


# encode 9
# br_id
data.cc = pd.Categorical(data.br_entry_status)
data['br_entry_statusencoded'] = data.cc.codes
data = data.drop('br_entry_status',axis=1)
print(data.br_entry_statusencoded.value_counts())



# encode 10
# br_id
data.cc = pd.Categorical(data.br_entry_status)
data['br_entry_statusencoded'] = data.cc.codes
data = data.drop('br_entry_status',axis=1)
print(data.br_entry_statusencoded.value_counts())




# encode 11
# br_id
data.cc = pd.Categorical(data.br_entry_status)
data['br_entry_statusencoded'] = data.cc.codes
data = data.drop('br_entry_status',axis=1)
print(data.br_entry_statusencoded.value_counts())



# encode 12
# br_id
data.cc = pd.Categorical(data.br_entry_status)
data['br_entry_statusencoded'] = data.cc.codes
data = data.drop('br_entry_status',axis=1)
print(data.br_entry_statusencoded.value_counts())



# encode 13
# br_id
data.cc = pd.Categorical(data.br_entry_status)
data['br_entry_statusencoded'] = data.cc.codes
data = data.drop('br_entry_status',axis=1)
print(data.br_entry_statusencoded.value_counts())



# encode 14
# br_id
data.cc = pd.Categorical(data.br_entry_status)
data['br_entry_statusencoded'] = data.cc.codes
data = data.drop('br_entry_status',axis=1)
print(data.br_entry_statusencoded.value_counts())


2    11083
1     2576
3     1015
0        4
Name: labelencoded, dtype: int64
76     475
643    278
573    142
373    131
652    123
      ... 
16       1
453      1
275      1
548      1
421      1
Name: busencoded, Length: 658, dtype: int64
77     879
23     571
64     457
88     441
72     355
      ... 
1       17
0       13
5       12
124      5
83       3
Name: point_aencoded, Length: 129, dtype: int64
64     1560
90      864
65      703
101     580
48      491
       ... 
99       14
97       12
23       12
62        4
38        3
Name: point_bencoded, Length: 147, dtype: int64
129    879
67     455
125    345
116    303
140    296
      ... 
4       15
14      14
15      12
13      12
70       8
Name: line_id_yencoded, Length: 167, dtype: int64
-1       12102
 23          4
 587         4
 846         4
 41          4
         ...  
 332         1
 383         1
 399         1
 453         1
 1041        1
Name: br_idencoded, Length: 2535, dtype: int64
-1    12102
 3     1591
 0

In [ ]:
# since the 'labelencoded' is not balanced, hence need to use SMOTE
# but some said SMOTE is not correct. i seems to agree too. 

In [351]:
from imblearn.over_sampling import SMOTE
seed = 100
k =1
X = data.loc[:, data.columns != 'labelencoded']
y = data.labelencoded
sm = SMOTE(sampling_strategy='auto', k_neighbors=k, random_state=seed)
X_res, y_res = sm.fit_resample(X, y)

ValueError: could not convert string to float: 'K1000'

In [ ]:
print(X_res.shape)
print(y_res.shape)

In [ ]:
X_res.head(2)

In [ ]:
df = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)
# rename the columns
df.to_csv('df_smoted.csv', index=False, encoding='utf-8')

In [ ]:
print(df.shape)
df.head(2)

# 2 Machine Learning

In [ ]:
from pycaret.classification import *
s = setup(data, target = 'labelencoded', session_id = 123,normalize = True, normalize_method = 'minmax')

In [ ]:
get_config('X_train_transformed').columns

In [ ]:
get_config('X_train_transformed')['bus_no'].hist()

In [ ]:
# # import ClassificationExperiment and init the class
# from pycaret.classification import ClassificationExperiment
# exp = ClassificationExperiment()

In [ ]:
# compare baseline models
best = compare_models()

In [ ]:
# plot confusion matrix
plot_model(best, plot = 'confusion_matrix')

In [ ]:
# plot AUC
plot_model(best, plot = 'auc')

In [ ]:
# plot feature importance
plot_model(best, plot = 'feature')

In [ ]:
# Ensemble model 
dt = create_model('dt')
# train individual models to blend
dt = create_model('dt', verbose = False)
rf = create_model('rf', verbose = False)
et = create_model('et', verbose = False)
# blend individual models
blender = blend_models(estimator_list = [dt, rf,et])

In [ ]:
blender

In [ ]:
stacked = stack_models(estimator_list = [dt, rf,et])

In [ ]:
# plot class report
plot_model(best, plot = 'class_report')

In [ ]:
save_model(best, 'Rapid_multiclassification_model')

# Predict Out-of-sample data 

In [ ]:
# copy data and drop Class variable

new_data = 
new_data.drop('labelencoded', axis=1, inplace=True)
new_data.head()

In [ ]:
# predict model on new_data
predictions = predict_model(best, data = new_data)
predictions.head()